In [ ]:
import pandas as pd
import numpy as np
import plotly
plotly.offline.init_notebook_mode(connected=False)
import plotly.graph_objs as go

# DVR

## DVI < DVI*

In [ ]:
Ts = np.arange(0, 50, .5)
Ls = np.arange(0, 24, .5)
Gv = 50    # assume 出穂に必要な最低日数、コシヒカリは 51.3 [d]
Th = 17.8   # assume  DVRが最低温度から半分になる温度、コシヒカリは 17.8 [℃]
Lc = 16   # 限界日長、コシヒカリは 16 [hr]
Gr = 30    # grain-filling の必要最低日数 、コシヒカリは 30.7 [d]

def plotly3d(df):
    data = [go.Scatter3d(x=df['T'], y=df['L'], z=df['DVR'], mode='markers', marker=dict(opacity=.6))]
    layout = go.Layout(
        margin=dict(l=0, r=0, b=0, t=0),
        scene = dict(
            xaxis=dict(title='T'),
            yaxis=dict(title='L'),
            zaxis=dict(title='DVR'),
        )
    )
    fig = go.Figure(data=data, layout=layout)
    return fig

In [ ]:
def calc_DVR1(T, L):
    A = 1     # assume
    tmp1 = 1 + np.exp(-A * (T - Th))
    tmp2 = Gv * tmp1
    return 1 / tmp2

df = pd.DataFrame([{'T': T, 'L': L, 'DVR': calc_DVR1(T, L)} for T in Ts for L in Ls])
plotly.offline.iplot(plotly3d(df))

In [ ]:
def calc_DVR2(T, L):
    A = 1     # assume
    B = 1     # assume
    if L > Lc:
        return 0
    tmp1 = 1 - np.exp(B * (L - Lc))
    tmp2 = Gv * (1 + np.exp(-A * (T - Th)))
    return tmp1 / tmp2

df = pd.DataFrame([{'T': T, 'L': L, 'DVR': calc_DVR2(T, L)} for T in Ts for L in Ls])
plotly.offline.iplot(plotly3d(df))

In [ ]:
def calc_DVR3(T, L):
    Kr = 0.071    # koshihikari
    Tcr = 6.4     # assume 
    tmp1 = np.exp(-Kr * (T - Tcr))
    return (1 - tmp1) / Gr

df = pd.DataFrame([{'T': T, 'L': L, 'DVR': calc_DVR3(T, L)} for T in Ts for L in Ls])
plotly.offline.iplot(plotly3d(df))

# CO2

CO2濃度が二倍になると出穂までの時間が4％減るという理論をもとに、Gvはこの式で決定される.

- Ca: CO2 concentration
- G: 350ppmのときのGv
- Gv: 出穂までの最低日数

In [ ]:
def calc_Gv(Ca):
    G = 1   # assume
    return G * (1 - 0.000114 * (Ca - 350))

Ca = np.arange(0, 2000, 10)
df = pd.DataFrame(dict(Ca=Ca, Gv=calc_Gv(Ca)))
ax = df.plot.scatter(x='Ca', y='Gv')

print(df[(df.Ca == 350) | (df.Ca == 700)])   # Gv が 4% 減る